# Generate the systems for multistates

In [2]:
#IMPORT
import os, sys, glob
import time

from collections import defaultdict
from pygromos.gromos.gromosPP import GromosPP
from pygromos.gromos.gromosXX import GromosXX
from pygromos.files import imd

import restraintmaker

#CHANGE HERE
gromos_bin_path = "/home/bschroed/Documents/code/gromosPP/installed/bin"
restraintmaker_path = os.path.abspath(os.path.dirname(restraintmaker.__file__)+"/..")

all ['_O6T', '_O71', 'G277', 'S002', 'M097', '8018', 'M030', '6KET', 'F313', '_P8I', 'M097', 'G078', 'M218', '6J29', 'G209', 'E1VB', 'TVVS', '_O70']
/home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/all_fifthteen/all_fifthteen.cnf
/home/bschroed/Documents/code/gromosPP/installed/bin/build_box @topo /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/all_fifthteen/all_fifthteen.top @pos /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/all_fifthteen/all_fifthteen.cnf @nsm 1 @dens 1.0 > /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/all_fifthteen/all_fifthteen_box.cnf 

/home/bschroed/Documents/code/gromosPP/installed/bin/sim_box @topo /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/al

## Path definitions
generate the subfolders for the systems - no changes required here.

In [ ]:
sets_dir = restraintmaker_path+"/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets"
mstate_dir = sets_dir+"/multistate"
pairwise_dir = sets_dir+"/pairwise"

resn_lib_path = sets_dir+"/resn_lib.lib"


gromPP = GromosPP(gromos_bin_path)
atb_dirs = restraintmaker_path+"/devtools/otherScripts/b_ATB_solvationFreeEnergies/ATB_molecules"
sys.path.append(atb_dirs+"/..")

sets_dir = restraintmaker_path+"/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets"
mstate_dir = sets_dir+"/multistate"

if(not os.path.exists(mstate_dir)):
    os.mkdir(mstate_dir)

##get all_single file_tops:
all_tops = glob.glob(atb_dirs+"/*/*top")
state_all_tops={os.path.basename(value).split(".")[0]: value for value in all_tops}
all_mstate_sys = glob.glob(mstate_dir+"/*")



## Build gromos files

In [ ]:
import utils_test_set_ATB as utils
all_states = utils.multistate_ligand_sets["all"]
all_combos = os.listdir(pairwise_dir)
all_combos = [x if(len(x)==2) else (x[0], "_"+x[2]) for x in list(map(lambda x: x.split("_"), all_combos))]
print(all_combos)
gromXX = GromosXX("/home/bschroed/Documents/code/gromosXX/installed/bin")
for state_a, state_b in all_combos:
        system_tops = [state_all_tops[x] for x in [state_a, state_b]]

        name = state_a+"_"+state_b
        out_dir = pairwise_dir+"/"+name
        out_prefix_path = out_dir+"/"+name
        print(name, [state_a, state_b])

        if(not os.path.exists(out_dir)):
            os.mkdir(out_dir)

        #build dualTop system
        out_top_path, out_ptp_path = gromPP.prep_eds(in_top_paths=system_tops, number_of_eds_states=len(system_tops), out_file_path=out_prefix_path)

        #generate cnf
        in_pdb = glob.glob(out_dir+"/*.pdb")[0]
        out_cnf_path = gromPP.pdb2gromos(in_pdb_path=in_pdb, in_top_path=out_top_path, in_lib_path=resn_lib_path,
                                         out_cnf_path=out_prefix_path+".cnf")

        #build posres/refpos
        from pygromos.files.coord import cnf
        cnf_file = cnf.Cnf(out_cnf_path)
        refpos_path,_ = cnf_file.write_refpos(out_prefix_path+".rfp")
        posresspec_path,_ = cnf_file.write_possrespec(out_prefix_path+".por", residues=list(cnf_file.residues.keys()))

        print(out_cnf_path)

## Emin vac

In [ ]:
        #vac - emin
        from pygromos.data.imd_templates import template_emin
        from pygromos.files.blocks import imd_blocks

        in_imd = imd.Imd(template_emin)
        in_imd.STEP.NSTLIM = 10
        in_imd.SYSTEM.NSM = 0
        in_imd.FORCE.NRE =[12,28]
        in_imd_path = in_imd.write(out_path=out_dir+"/vac_emin.imd")

        gromXX.md_run(in_coord_path=out_cnf_path, in_topo_path=out_top_path, in_imd_path=in_imd_path,out_prefix=out_dir+"/emin",
                      in_pert_topo_path=out_ptp_path, in_refpos_path=refpos_path, in_posresspec_path=posresspec_path)
                      #in_disres_path=out_prefix_path+".disres")

        break

